In [1]:
from numpy.random import MT19937
from numpy.random import RandomState, SeedSequence

index_pairs = list(itertools.combinations())

n-1
n-2
n-3
n-4
...
2

In [2]:
import numpy as np
import os
from scipy.special import comb
import itertools
import math
import matplotlib.pyplot as plt


def mnist(b, mode, n2, eta0, maxIter, valSize, testSize, seed):
    b = float(b)
    n2 = int(n2)
    eta0 = float(eta0)
    maxIter = int(maxIter)
    testSize = int(testSize)
    valSize = int(valSize)
    seed = int(seed)
    iterRisk = (
        list(range(1, 101, 10))
        + list(range(100, 1001, 100))
        + list(range(1000, 10001, 1000))
    )
    # iterRisk = list(range(1, maxIter + 1, 100))

    XTr, YTr, XVal, YVal, XTe, YTe = load_mnist_data()
    # nTrain = XTr.shape[0]

    # Set random seed for reproducibility
    # rs = RandomState(MT19937(SeedSequence(seed)))
    if mode == "i":
        np.random.seed(seed)
    elif mode == "p":
        np.random.seed(seed + 1000)
    else:
        raise ValueError("Mode needs to be i or p")

    # Validation set
    nVal = XVal.shape[0]
    pairsVal = np.zeros((valSize, 2), dtype=int)
    pairsVal[:, 0] = np.random.choice(nVal, valSize, replace=True)
    pairsVal[:, 1] = np.random.choice(nVal, valSize, replace=True)
    pYVal = -np.ones(valSize)
    pYVal[YVal[pairsVal[:, 0]] == YVal[pairsVal[:, 1]]] = 1
    diffVal = XVal[pairsVal[:, 0]] - XVal[pairsVal[:, 1]]

    # Test set
    nTest = XTe.shape[0]
    pairsTest = np.zeros((testSize, 2), dtype=int)
    pairsTest[:, 0] = np.random.choice(nTest, testSize, replace=True)
    pairsTest[:, 1] = np.random.choice(nTest, testSize, replace=True)
    pYTe = -np.ones(testSize)
    pYTe[YTe[pairsTest[:, 0]] == YTe[pairsTest[:, 1]]] = 1
    diffTe = XTe[pairsTest[:, 0]] - XTe[pairsTest[:, 1]]

    saveDir = f"results/real/bias{b}_mode{mode}_n2{n2}_eta0{eta0}_maxIter{maxIter}_valSize{valSize}_testSize{testSize}_{seed}"
    os.makedirs(saveDir, exist_ok=True)

    # Initialize RNG based on mode
    if mode == "p":
        np.random.seed(seed)
    elif mode == "i":
        np.random.seed(seed + 10)
    else:
        raise ValueError("Invalid mode")

    # Run the SGD algorithm
    res = sgd_pairs(
        XTr,
        YTr,
        diffVal,
        pYVal,
        diffTe,
        pYTe,
        b,
        mode,
        n2,
        eta0,
        maxIter,
        iterRisk,
        saveDir,
    )


def sgd_pairs(
    X, Y, diffVal, pYVal, diffTe, pYTe, b, mode, n2, eta0, maxIter, iterRisk, saveDir
):
    n, d = X.shape
    # M = np.eye(d) + 0.01 * np.random.randn(
    #     d, d
    # )  # Initialize M as identity matrix + some noise
    # M = np.random.randn(d, d)
    M = np.eye(d)
    W = np.ones(n)
    n_pairs = n(n-1)/2
    Wp = np.ones(n_pairs)
    countRisk = 0

    # Number of pairs in mini-batch
    m = int(comb(n2, 2))

    results = []

    for k in range(1, maxIter + 1):
        if mode == "per":
            # Sample m pairs with replacement
            pairs = np.zeros((m, 2), dtype=int)

            pairs[:, 0] = np.random.choice(n, m, replace=True)
            pairs[:, 1] = np.random.choice(n, m, replace=True)
        elif mode == "ier":
            # Sample instances and compute combinations
            s = np.random.choice(n, n2, replace=True)
            pairs = np.array(list(itertools.combinations(s, 2)))
        elif mode == "iu":
            # Sample instances with unequal probabilities and compute combinations
            s = np.random.choice(n, n2, replace=False, p=W)
            pairs = np.array(list(itertools.combinations(s, 2)))
        elif mode == "pu":
            # Sample m pairs with unequal probabilities
            pairs = np.zeros((m, 2), dtype=int)
            pair_indice = np.random.choice(n_pairs, m, replace=False, p=Wp)
            
            pairs[:, 0] = 
        elif mode == "ie":
            # Sample instances without replacement
            s = np.random.choice(n, n2, replace=False)
            pairs = np.array(list(itertools.combinations(s, 2)))
        elif mode == "pe":
            # Sample m pairs without replacement
            pairs = np.zeros((m, 2), dtype=int)

            pairs[:, 0] = np.random.choice(n, m, replace=False)
            pairs[:, 1] = np.random.choice(n, m, replace=False)
        else:
            raise ValueError("Not the right mode.")

        # Compute pair labels
        pY = -np.ones(m)
        # pY = np.zeros((m, m))
        pY[Y[pairs[:, 0]] == Y[pairs[:, 1]]] = 1

        # Compute gradients
        gradM = np.zeros((d, d))
        diff = X[pairs[:, 0]] - X[pairs[:, 1]]
        slack = (
            1 + pY * (np.sum((M @ diff.T) * diff.T, axis=0) - b)
        ) > 0  # diff @ M @ diff.T
        for i in range(m):
            if slack[i]:
                gradM += pY[i] * np.outer(diff[i], diff[i])

        gradM /= m

        # Update M with decreasing step size
        # M -= eta0 * (1 / k) * gradM
        M -= eta0 * math.sqrt(1 / k) * gradM

        if k in iterRisk:
            # Project M onto the PSD cone
            eigvals, eigvecs = np.linalg.eigh(M)
            eigvals[eigvals < 0] = 0
            M2 = eigvecs @ np.diag(eigvals) @ eigvecs.T
            # plt.imshow(M2)
            # plt.colorbar()
            # plt.savefig(os.path.join(saveDir, f"M_{k}.png"))
            # plt.close()

            # Compute validation and test risks
            RVal = compute_risk_pairs(diffVal, pYVal, b, M2)
            RTe = compute_risk_pairs(diffTe, pYTe, b, M2)
            RTr = compute_risk_pairs(diff, pY, b, M2)

            results.append([k, RVal, RTe])
            print(
                f"iter {k}, train_risk {RTr:.5f}, val risk {RVal:.5f}, test risk {RTe:.5f}"
            )
            np.save(os.path.join(saveDir, f"M_{k}.npy"), M2)

    np.save(os.path.join(saveDir, "res.npy"), results)
    return results


def compute_risk_pairs(diffTe, pYTe, b, M):
    slack = np.maximum(0, 1 + pYTe * (np.sum((M @ diffTe.T) * diffTe.T, axis=0) - b))
    return np.mean(slack)


def load_mnist_data():
    data = np.load("data/data_50000_10000_10000.npz")
    XTr = data["X_train"]
    YTr = data["y_train"]
    XVal = data["X_val"]
    YVal = data["y_val"]
    XTe = data["X_test"]
    YTe = data["y_test"]
    return XTr, YTr, XVal, YVal, XTe, YTe

In [3]:
import numpy as np
import os
import matplotlib.pyplot as plt


def make_res2(b, n2, eta0, maxIter, valSize, testSize, runs):
    # Load first run for 'i' mode
    path = f"results/real/bias{b}_modei_n2{n2}_eta0{eta0}_maxIter{maxIter}_valSize{valSize}_testSize{testSize}_0/res.npy"
    resI = np.load(path)
    resITr = resI[:, [0, 1]]
    resITe = resI[:, [0, 2]]

    # Load subsequent runs for 'i' mode
    for s in range(1, runs):
        path = f"results/real/bias{b}_modei_n2{n2}_eta0{eta0}_maxIter{maxIter}_valSize{valSize}_testSize{testSize}_{s}/res.npy"
        resI = np.load(path)
        resITr = np.column_stack((resITr, resI[:, 1]))
        resITe = np.column_stack((resITe, resI[:, 2]))

    # Load first run for 'p' mode
    path = f"results/real/bias{b}_modep_n2{n2}_eta0{eta0}_maxIter{maxIter}_valSize{valSize}_testSize{testSize}_0/res.npy"
    resP = np.load(path)
    resPTr = resP[:, [0, 1]]
    resPTe = resP[:, [0, 2]]

    # Load subsequent runs for 'p' mode
    for s in range(1, runs):
        path = f"results/real/bias{b}_modep_n2{n2}_eta0{eta0}_maxIter{maxIter}_valSize{valSize}_testSize{testSize}_{s}/res.npy"
        resP = np.load(path)
        resPTr = np.column_stack((resPTr, resP[:, 1]))
        resPTe = np.column_stack((resPTe, resP[:, 2]))

    # Clear temporary res arrays
    del resI, resP

    # Compute means and standard deviations
    meanITr = np.mean(resITr[:, 1:], axis=1)
    meanITe = np.mean(resITe[:, 1:], axis=1)
    meanPTr = np.mean(resPTr[:, 1:], axis=1)
    meanPTe = np.mean(resPTe[:, 1:], axis=1)
    stdITr = np.std(resITr[:, 1:], axis=1)
    stdITe = np.std(resITe[:, 1:], axis=1)
    stdPTr = np.std(resPTr[:, 1:], axis=1)
    stdPTe = np.std(resPTe[:, 1:], axis=1)

    # Plot training results
    plt.plot(resPTr[:, 0], meanPTr, "b", linewidth=1, label="Pair sampling")
    plt.fill_between(
        resPTr[:, 0], meanPTr - stdPTr, meanPTr + stdPTr, color="b", alpha=0.2
    )
    # plt.plot(resPTr[:, 0], meanPTr + stdPTr, "b", "--", linewidth=1)
    # plt.plot(resPTr[:, 0], meanPTr - stdPTr, "b", "--", linewidth=1)
    plt.plot(resITr[:, 0], meanITr, "r", linewidth=1, label="Naive sampling")
    plt.fill_between(
        resITr[:, 0], meanITr - stdITr, meanITr + stdITr, color="r", alpha=0.2
    )
    # plt.plot(resITr[:, 0], meanITr + stdITr, "r", "--", linewidth=1)
    # plt.plot(resITr[:, 0], meanITr - stdITr, "r", "--", linewidth=1)
    plt.xlabel("Iterations")
    plt.ylabel("Hinge Loss")
    plt.legend()
    plt.savefig(
        f"results/real/plot_train_bias{b}_n2{n2}_eta0{eta0}_maxIter{maxIter}_valSize{valSize}_testSize{testSize}_runs{runs}_meanstdTr.png",
        format="png",
    )
    plt.clf()

    # Plot test results
    plt.plot(resPTe[:, 0], meanPTe, "b", linewidth=1, label="Pair sampling")
    plt.fill_between(
        resPTe[:, 0], meanPTe - stdPTe, meanPTe + stdPTe, color="b", alpha=0.2
    )
    # plt.plot(resPTe[:, 0], meanPTe + stdPTe, "b", linewidth=1)
    # plt.plot(resPTe[:, 0], meanPTe - stdPTe, "b", linewidth=1)
    plt.plot(resITe[:, 0], meanITe, "r", linewidth=1, label="Naive sampling")
    plt.fill_between(
        resITe[:, 0], meanITe - stdITe, meanITe + stdITe, color="r", alpha=0.2
    )
    # plt.plot(resITe[:, 0], meanITe + stdITe, "r", linewidth=1)
    # plt.plot(resITe[:, 0], meanITe - stdITe, "r", linewidth=1)
    plt.title("Test loss mean and standard deviation per iteration")
    plt.xlabel("Iterations")
    plt.ylabel("Hinge Loss")
    plt.legend()
    plt.savefig(
        f"results/real/plot__test_bias{b}_n2{n2}_eta0{eta0}_maxIter{maxIter}_valSize{valSize}_testSize{testSize}_runs{runs}_meanstdTe.png",
        format="png",
    )
    plt.clf()

    # Plot individual training results
    plt.plot(resPTr[:, 0], resPTr[:, 1], "b", linewidth=1, label="Pair sampling")
    plt.plot(resITr[:, 0], resITr[:, 1], "r", linewidth=1, label="Naive sampling")
    for s in range(2, runs + 1):
        plt.plot(resPTr[:, 0], resPTr[:, s], "b", linewidth=1)
        plt.plot(resITr[:, 0], resITr[:, s], "r", linewidth=1)
    plt.title("Individual training losses per iteration")
    plt.xlabel("Iterations")
    plt.ylabel("Hinge Loss")
    plt.legend()
    plt.savefig(
        f"results/real/plot_ind_train_bias{b}_n2{n2}_eta0{eta0}_maxIter{maxIter}_valSize{valSize}_testSize{testSize}_runs{runs}_indTr.png",
        format="png",
    )
    plt.clf()

    # Plot individual test results
    plt.plot(resPTe[:, 0], resPTe[:, 1], "b", linewidth=1, label="Pair sampling")
    plt.plot(resITe[:, 0], resITe[:, 1], "r", linewidth=1, label="Naive sampling")
    for s in range(2, runs + 1):
        plt.plot(resPTe[:, 0], resPTe[:, s], "b", linewidth=1)
        plt.plot(resITe[:, 0], resITe[:, s], "r", linewidth=1)
    plt.title("Individual test losses per iteration")
    plt.xlabel("Iterations")
    plt.ylabel("Hinge Loss")
    plt.legend()
    plt.savefig(
        f"results/real/plot_ind_test_bias{b}_n2{n2}_eta0{eta0}_maxIter{maxIter}_valSize{valSize}_testSize{testSize}_runs{runs}_indTe.png",
        format="png",
    )
    plt.clf()


def load_results(path):
    # Placeholder for the actual loading function
    return np.load(path)

In [4]:
b = 2.1  # Example bias value
eta0 = 0.01  # Initial step size
maxIter = 5000  # Maximum number of iterations
valSize = 1000  # Validation size
testSize = 1000  # Test size
runs = 50  # Number of runs

In [5]:
n2s = [5]

In [6]:
for n2 in n2s:
    value = n2 * (n2 - 1) / 2
    print(value)

10.0


In [7]:
import shutil

shutil.rmtree("results/real", ignore_errors=True)
os.makedirs("results/real", exist_ok=True)

In [8]:
for i in range(runs):
    print(f"Run number {i}")
    print("Naive sampling")
    for n2 in n2s:
        print(f"Sample size {n2}")
        mnist(b, "i", n2, eta0, maxIter, valSize, testSize, seed=i)
    print("Pair sampling")
    for n2 in n2s:
        j = np.random.randint(1, 10)
        print(f"Sample size {n2}")
        mnist(b, "p", n2, eta0, maxIter, valSize, testSize, seed=i)

Run number 0
Naive sampling
Sample size 5
iter 1, train_risk 1.06137, val risk 0.99361, test risk 0.99930
iter 11, train_risk 0.94667, val risk 0.99004, test risk 0.99534
iter 21, train_risk 1.09554, val risk 0.98770, test risk 0.99283
iter 31, train_risk 1.10057, val risk 0.98546, test risk 0.99042
iter 41, train_risk 1.04807, val risk 0.98368, test risk 0.98851
iter 51, train_risk 1.08517, val risk 0.98210, test risk 0.98677
iter 61, train_risk 0.91574, val risk 0.98090, test risk 0.98547
iter 71, train_risk 1.06963, val risk 0.97957, test risk 0.98402
iter 81, train_risk 0.93080, val risk 0.97831, test risk 0.98264
iter 91, train_risk 0.93029, val risk 0.97713, test risk 0.98133
iter 100, train_risk 0.95130, val risk 0.97644, test risk 0.98058
iter 200, train_risk 0.86579, val risk 0.96798, test risk 0.97143
iter 300, train_risk 1.34140, val risk 0.96165, test risk 0.96463
iter 400, train_risk 1.01912, val risk 0.95592, test risk 0.95840
iter 500, train_risk 0.88915, val risk 0.9512

In [9]:
for n2 in n2s:
    make_res2(b, n2, eta0, maxIter, valSize, testSize, runs)

<Figure size 640x480 with 0 Axes>